In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

## South Korea

In [6]:
response = requests.get('https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_South_Korea')
soup = BeautifulSoup(response.content, 'lxml')
jan = soup.find_all('tr', attrs={'id': 'mw-customcollapsible-jan'})
feb = soup.find_all('tr', attrs={'id': 'mw-customcollapsible-feb'})
march = soup.find_all('tr', attrs={'id': 'mw-customcollapsible-mar'})
april = soup.find_all('tr', attrs={'id': 'mw-customcollapsible-apr'})
april_2 = soup.find_all('tr', attrs={'id': 'mw-customcollapsible-apr-l15'})

In [9]:
df_dict = dict()
df_dict['date'] = list()
df_dict['cases'] = list()
df_dict['deaths'] = list()
df_dict['recovered'] = list()

def populate_dict(dataset):
    for data in dataset:
        date = data.find('td').get_text().replace(',', '')
        cases = data.find_all('td')[2].find('span', attrs={'class':'cbs-ibr'}).get_text().replace(',', '')
        deaths = data.find_all('td')[3].find('span', attrs={'class':'cbs-ibr'}).get_text().replace(',', '')
        div_ = data.find_all('div')
        if len(div_) >= 3:
            recovered = div_[1].get('title').replace(',', '')
        else:
            recovered = '0'
        df_dict['date'].append(date)
        df_dict['cases'].append(cases)
        df_dict['deaths'].append(deaths)
        df_dict['recovered'].append(recovered)
populate_dict(jan)
populate_dict(feb)
populate_dict(march)
populate_dict(april)
populate_dict(april_2)

In [34]:
south_korea = pd.DataFrame(df_dict)
def populate_df(df):
    df['date'] = pd.to_datetime(df['date'])
    df['recovered'] = df['recovered'].replace('', '0').astype(int)
    df['cases'] = df['cases'].replace('', '0').astype(int)
    df['deaths'] = df['deaths'].replace('', '0').astype(int)
    df['active'] = df['cases'] - df['recovered'] - df['deaths']

    df = df.set_index('date')
    return df

In [35]:
south_korea = south_korea[~(south_korea['date'] == '⋮')]

south_korea = populate_df(south_korea)

south_korea['date'] = pd.to_datetime(south_korea['date'])

idx = pd.date_range('01-20-2020', '04-20-2020')

south_korea = south_korea.reindex(idx)

south_korea = south_korea.fillna(method='ffill')

### save df

In [49]:
south_korea.to_csv('data/master_data/south_korea/from_wiki.csv')

## New York

In [50]:
response = requests.get('https://en.wikipedia.org/wiki/2020_coronavirus_pandemic_in_New_York_(state)')
soup = BeautifulSoup(response.content, 'lxml')
march = soup.find_all('tr', attrs={'id': 'mw-customcollapsible-mar'})
april = soup.find_all('tr', attrs={'id': 'mw-customcollapsible-apr'})
april_2 = soup.find_all('tr', attrs={'id': 'mw-customcollapsible-apr-l15'})

In [55]:
df_dict = dict()
df_dict['date'] = list()
df_dict['cases'] = list()
df_dict['deaths'] = list()
df_dict['recovered'] = list()

populate_dict(march)
populate_dict(april)
populate_dict(april_2)

In [56]:
new_york = pd.DataFrame(df_dict)

In [57]:
new_york = populate_df(new_york)

### Save df

In [59]:
new_york.to_csv('data/master_data/new_york/from_wiki.csv')